#### <font color="purple">Препроцессинг данных</font>

Препроцессинг (preprocessing, предварительная обработка) данных — это один из самых важных этапов ML-пайплайна (конвейера машинного обучения). Качество обработки даннных влияет на качество обучения модели.

Этапы препроцессинга:
- <font color="purple">Заполнение пропусков в данных.</font> Используйте для этого такие методы, как подстановка среднего или модального значения вместо пропущенных значений, или более совершенные подходы — вроде восстановления пропущенных значений с использованием метода K-ближайших соседей (K-Nearest Neighbors, KNN).
- <font color="purple">Нормализация/стандартизация значений признаков.</font> Есть алгоритмы чуствительные к масштабу признаков, особенно когда в данных очень большой разброс от максимального до минимального значения, в таких случаях даже небольшая ошибка в предсказании очень сильно влияет на качество. 
- <font color="purple">Кодировка категориальных (строковых) переменных.</font> Преобразуйте категориальные переменные в числовые значения, используя такие методы, как одноразовое кодирование (one-hot encoding) или кодирование меток (label encoding). Также это можно делать и в ручную с помощью map(), однако если присваивать обычные цифры, модели могут счесть это за порядок (ранг/место) а не категорию/класс.
- <font color="purple">Разбивайте данные на обучающую и тестовую выборки до препроцессинга.</font> Разбивайте данные на обучающую и тестовую выборки до их обработки, проводимой с использованием каких-либо методик препроцессинга данных. Это позволит исключить утечку данных.

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Staвыборки
X_train, X_test, y_train, y_test = traindardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Прочесть данные из CSV-файла
data = pd.read_csv('your_data.csv')

# Указать имя столбца, в котором находится целевая переменная
target_column = 'target'

# Разделить данные на признаки и целевую переменную
X = data.drop(target_column, axis=1)
y = data[target_column]

# Разделить данные на обучающую и тестовую n_test_split(X, y, test_size=0.20, random_state=10)

# Идентифицировать столбцы, содержащие числовые и категориальные переменные
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Определить процедуру препроцессинга для числовых значений
numeric_transformer = Pipeline(steps=[
	('imputer', SimpleImputer(strategy='mean')),
	('scaler', StandardScaler())
])

# Определить процедуру препроцессинга для категориальных признаков
categorical_transformer = Pipeline(steps=[
	('encoder', OneHotEncoder(drop='first'))
])

# Скомбинировать процедуры препроцессинга данных
preprocessor = ColumnTransformer(
	transformers=[
    	('num', numeric_transformer, numeric_features),
    	('cat', categorical_transformer, categorical_features)
	]
)

# Применить процедуру препроцессинга к обучающим данным
X_train_processed = preprocessor.fit_transform(X_train)

# Применить процедуру препроцессинга к тестовым данным
X_test_processed = preprocessor.transform(X_test)

Последовательности процедур, применяемых в ходе препроцессинга данных, определяются отдельно для числовых и категориальных признаков. А именно, при обработке числовых признаков мы восстанавливаем отсутствующие значения, используя среднее, после чего масштабируем значения, используя StandardScaler. Категориальные признаки мы обрабатываем, применяя одноразовое кодирование. Эти процедуры препроцессинга мы комбинируем, используя ColumnTransformer, после чего применяем их по-отдельности к обучающей и тестовой выборкам, что позволяет избежать утечки данных.

#### <font color="purple">Конструирование признаков</font>
Конструирование признаков (Feature Engineering) — это систематический процесс модификации существующих признаков и создания новых. Делается это для того, чтобы повысить качество работы модели.
Основные приемы конструирования признаков:
- <font color="purple">Создание признаков, отражающих взаимотношения других признаков. </font> Признаки, отражающие взаимоотношения других признаков (Interaction Features) способны выявить скрытые закономерности, которые могут не раскрыться при работе с отдельными переменными. Предположим, у нас имеются признаки price (цена) и qty_sold (количество проданных товаров). В качестве признака, отражающего взаимодействие других признаков, может выступать произведение двух исходных переменных, показывающее общий объём продажи товара:

In [ ]:
# Создать признак, отражающий взаимоотношения других признаков
data['price_qty_interaction'] = data['price'] * data['qty_sold']

- <font color="purple">Разделение признаков, содержащих информацию о дате и времени.</font> Данные, содержащие сведения о дате и времени, можно разобрать на информативные компоненты — такие, как год, месяц, день и день недели. Эти компоненты способны помочь в выявлении временных закономерностей, присутствующих в данных.


In [ ]:
# Извлечь признаки из date
data['date'] = pd.to_datetime(['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day_of_week'] = data['date'].dt.dayofweek

- <font color="purple">Биннинг (группировка).</font> Биннинг -  это метод проектирования признаков, который объединяет различные числовые поддиапазоны в интервалы или бары . Во многих случаях группировка преобразует числовые данные в категориальные. Пусть имеется признак income (доход). Мы можем создать «контейнеры», позволяющие разделить данные о доходе на три категории: Low, Medium, High (низкий, средний и высокий доход):

In [ ]:
# Биннинг признаков, представленных непрерывными значениями
data['income_bin'] = pd.cut(data['income'], bins=3, labels=['Low', 'Medium', 'High'])

#### <font color="purple">Дислабаланс классов</font> 
https://imbalanced-learn.org/stable/under_sampling.html

Дисбаланс классов (Class Imbalance) — это распространённая проблема, возникающая при работе с наборами данных, собранными в реальном мире. Это — ситуация, когда в целевой переменной классы представлены неравномерно. Метрики эффективности работы модели, которая обучена на несбалансированных наборах данных, нельзя назвать надёжными.

Методы решения проблем с дисбалансом:
- <font color="purple">Увеличение выборки (Oversampling).</font> Увеличение количества экземпляров в классе, представленном недостаточным количеством образцов (minority class, «малый класс»). Делается это путём дублирования элементов или путём создания синтетических образцов. Популярным методом для генерирования искусственных образцов является SMOTE (Synthetic Minority Over-sampling Technique, метод синтетической передискретизации меньшинства).

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X = data.drop('target', axis=1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

# Применить SMOTE
smote = SMOTE(random_state=10)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

- <font color="purple">Уменьшение выборки (Undersampling).</font> Уменьшение количества экземпляров в классе, представленном избыточным количеством образцов, путём случайного удаления некоторых из них. RandomUnderSampler (Случайный undersampling). Этот метод случайно удаляет образцы из мажоритарного класса до достижения баланса:

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Применение undersampling
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

NearMiss (Undersampling на основе ближайших соседей)
Этот метод имеет версии (version=1,2,3). Он удаляет образцы мажоритарного класса, основываясь на расстояниях до миноритарного класса (например, version=1 сохраняет образцы, близкие к миноритарным). 

NearMiss-2 отбирает положительные образцы, для которых среднее расстояние до N Наиболее удаленные образцы отрицательного класса являются наименьшими. 

NearMiss-3 — это двухэтапный алгоритм. Во-первых, для каждого отрицательного образца определяется их М будут сохранены ближайшие соседи. Затем будут отобраны положительные образцы, для которых среднее расстояние до N метод ближайших соседей является самым большим.



In [ ]:
from imblearn.under_sampling import NearMiss

# Применение NearMiss (version=1)
nm = NearMiss(version=1, n_neighbors=3)
X_resampled, y_resampled = nm.fit_resample(X_train, y_train)

TomekLinks (Удаление Tomek-ссылок). Метод удаляет пары ближайших соседей из разных классов (Tomek links), очищая границы между классами. Полезен для удаления шума.  Основная идея заключается в том, что связи Томека представляют собой шумные или трудноклассифицируемые наблюдения и не помогут алгоритму найти подходящую границу дискриминации.

In [ ]:
from imblearn.under_sampling import TomekLinks

# Применение TomekLinks
tl = TomekLinks(sampling_strategy='auto')
X_resampled, y_resampled = tl.fit_resample(X_train, y_train)